# MNIST Training using PyTorch and Horovod

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

Horovod is a distributed deep learning training framework for TensorFlow, Keras, PyTorch, and MXNet. This notebook example shows how to:
- use Horovod with PyTorch
- run distributed parallel training
- leverage Spot instances
- save checkpoints.

You may have to raise the number of parallel spot instances in your account to be able to run it (the code uses **6 p3.2xlarge instances in parallel** during the hyperparameter optimization).

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.


---

## Setup

_This notebook was created and tested on ml.p3.2xlarge notebook instances._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with the appropriate full IAM role arn string(s).


In [1]:
from datetime import datetime

start_time = datetime.now()

In [2]:
import sagemaker
import time
import numpy as np
import boto3
import pandas as pd

from torchvision import datasets, transforms
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from IPython.display import HTML

sagemaker_session = sagemaker.Session()

bucket = 'sagemaker-pytorch-dist-demo' # or use `sagemaker_session.default_bucket()`
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

## Data
### Getting the data

In this example, we will ues MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

In [3]:
data_start_time = datetime.now()

In [4]:
datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [5]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print(f'input spec (in this case, just an S3 path): {inputs}')

input spec (in this case, just an S3 path): s3://sagemaker-pytorch-dist-demo/sagemaker/DEMO-pytorch-mnist


## Train
### Training script
The `mnist_ckppoint.py` script provides the code we need for training a SageMaker model.
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

This script uses Horovod framework for distributed training where Horovod-related lines are commented with `Horovod:`. For example, `hvd.broadcast_parameters`, `hvd.DistributedOptimizer` and etc.

For example, the script run by this notebook:

In [6]:
!pygmentize code/mnist_ckpoint.py

#from __future__ import print_function
import json
import logging
import os
import sys
import argparse
import horovod.torch as hvd
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data.distributed

from torchvision import datasets, transforms
from torch.nn import NLLLoss
from model_def import Net

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


def _get_train_data_loader(batch_size, training_dir, **kwargs):
    logger.info("Get train data sampler and data loader")
    dataset = datasets.MNIST(training_dir, train=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))
    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset, num_replicas=hvd.size(), rank=hvd.rank())
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, **kwargs)
    return train_lo

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.p3.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist_ckpoint.py` script above.


In [7]:
train_start_time = datetime.now()

In [8]:
metrics = [
    {
        "Name": "Test Accuracy",
        "Regex": "Test set:.+Accuracy: (\d+(?:\.\d+))%"
    },
    {
        "Name": "Test Loss",
        "Regex": "Test set:.+Average loss: (\d+(?:\.\d+)),.+" 
    },
    {
        "Name": "Epoch",
        "Regex": 'Train Epoch: (\d+)'
    },
    {
        "Name": "Epoch completion",
        "Regex": 'Train Epoch: \d+ \[\d+/\d+ \((\d+%)\)\]'
    },
    {
        "Name": "Train loss",
        "Regex": 'Train Epoch: \d+ \[\d+/\d+ \(\d+%\)\] Loss: (\d+\.\d+)'
    },
]

estimator = PyTorch(entry_point='mnist_ckpoint.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.3.1',
                    train_instance_type='ml.p3.2xlarge',
                    metric_definitions=metrics,
                    train_use_spot_instances=True,
                    train_max_wait=25*60*60,
                    train_instance_count=2,
                    checkpoint_s3_uri=f's3://{bucket}/checkpoints',
                    hyperparameters={
                        'epochs': 5,
                        'backend': 'nccl',
                        'test-checkpoint': 4
                    })

In order to demonstrate the checkpoint resume capability, we'll tell the script to raise an exception at the beginning of the fourth epoch (hyperparameter `'test-checkpoint': 4`) above.

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.

**The next cell is designed to fail at the 4th epoch. When it does, just resume the notebook execution from the one after it**.


In [9]:
estimator.fit({'training': inputs}, wait=True)

2020-02-17 11:14:06 Starting - Starting the training job...
2020-02-17 11:14:08 Starting - Launching requested ML instances......
2020-02-17 11:15:16 Starting - Preparing the instances for training......
2020-02-17 11:16:20 Downloading - Downloading input data...
2020-02-17 11:16:53 Training - Downloading the training image.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-17 11:18:10,314 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-17 11:18:10,339 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-17 11:18:13,059 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-17 11:18:13,084 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.



2020-02-17 11:18:12 Training - Training image download completed. Training in progress.INFO:__main__:Train Epoch: 1 [640/60000 (1%)] Loss: 2.304615
Starting training from the beginning
INFO:__main__:Train Epoch: 1 [640/60000 (1%)] Loss: 2.304614
Starting training from the beginning
INFO:__main__:Train Epoch: 1 [1280/60000 (2%)] Loss: 2.312132
Test checkpoint is 4
INFO:__main__:Train Epoch: 1 [1280/60000 (2%)] Loss: 2.312131
Test checkpoint is 4
INFO:__main__:Train Epoch: 1 [1920/60000 (3%)] Loss: 2.291755
Get train data sampler and data loader
INFO:__main__:Train Epoch: 1 [2560/60000 (4%)] Loss: 2.294058
Get test data sampler and data loader
INFO:__main__:Train Epoch: 1 [3200/60000 (5%)] Loss: 2.291983
Processes 60000/60000 (100%) of train data
INFO:__main__:Train Epoch: 1 [3840/60000 (6%)] Loss: 2.259225
Processes 10000/10000 (100%) of test data
INFO:__main__:Train Epoch: 1 [4480/60000 (7%)] Loss: 2.270091
[2020-02-17 11:18:23.525 algo-1:46 INFO json_config.py:90] Creating hook from 

INFO:__main__:Train Epoch: 1 [42240/60000 (70%)] Loss: 0.578561
Train Epoch: 1 [35840/60000 (60%)] Loss: 0.681572
INFO:__main__:Train Epoch: 1 [42880/60000 (71%)] Loss: 0.652027
Train Epoch: 1 [36480/60000 (61%)] Loss: 0.702572
INFO:__main__:Train Epoch: 1 [43520/60000 (72%)] Loss: 0.670639
Train Epoch: 1 [37120/60000 (62%)] Loss: 0.947972
INFO:__main__:Train Epoch: 1 [44160/60000 (74%)] Loss: 0.734596
Train Epoch: 1 [37760/60000 (63%)] Loss: 0.586345
INFO:__main__:Train Epoch: 1 [44800/60000 (75%)] Loss: 0.658686
Train Epoch: 1 [38400/60000 (64%)] Loss: 0.636209
INFO:__main__:Train Epoch: 1 [45440/60000 (76%)] Loss: 0.435589
Train Epoch: 1 [39040/60000 (65%)] Loss: 0.670400
INFO:__main__:Train Epoch: 1 [46080/60000 (77%)] Loss: 0.610422
Train Epoch: 1 [39680/60000 (66%)] Loss: 0.831303
INFO:__main__:Train Epoch: 1 [46720/60000 (78%)] Loss: 0.688221
Train Epoch: 1 [40320/60000 (67%)] Loss: 0.694420
INFO:__main__:Train Epoch: 1 [43520/60000 (72%)] Loss: 0.695250
Train Epoch: 1 [37120/60

INFO:__main__:Train Epoch: 2 [19200/60000 (32%)] Loss: 0.365597
Train Epoch: 2 [13440/60000 (22%)] Loss: 0.494417
INFO:__main__:Train Epoch: 2 [19840/60000 (33%)] Loss: 0.436345
Train Epoch: 2 [14080/60000 (23%)] Loss: 0.446381
INFO:__main__:Train Epoch: 2 [20480/60000 (34%)] Loss: 0.180099
Train Epoch: 2 [14720/60000 (25%)] Loss: 0.173765
INFO:__main__:Train Epoch: 2 [21120/60000 (35%)] Loss: 0.326352
Train Epoch: 2 [15360/60000 (26%)] Loss: 0.329959
INFO:__main__:Train Epoch: 2 [21760/60000 (36%)] Loss: 0.598429
Train Epoch: 2 [16000/60000 (27%)] Loss: 0.550346
INFO:__main__:Train Epoch: 2 [22400/60000 (37%)] Loss: 0.408572
Train Epoch: 2 [16640/60000 (28%)] Loss: 0.481258
INFO:__main__:Train Epoch: 2 [23040/60000 (38%)] Loss: 0.445631
Train Epoch: 2 [17280/60000 (29%)] Loss: 0.294725
INFO:__main__:Train Epoch: 2 [20480/60000 (34%)] Loss: 0.167429
Train Epoch: 2 [14720/60000 (25%)] Loss: 0.186437
INFO:__main__:Train Epoch: 2 [21120/60000 (35%)] Loss: 0.304395
Train Epoch: 2 [15360/60

INFO:__main__:Epoch: 2#011Test set: Average loss: 0.1267, Accuracy: 96.08%
Train Epoch: 2 [54400/60000 (91%)] Loss: 0.278081

Train Epoch: 2 [55040/60000 (92%)] Loss: 0.290665
INFO:__main__:Epoch: 2#011Test set: Average loss: 0.1267, Accuracy: 96.09%
Train Epoch: 2 [54400/60000 (91%)] Loss: 0.269703

Train Epoch: 2 [55040/60000 (92%)] Loss: 0.278394
INFO:__main__:Train Epoch: 3 [640/60000 (1%)] Loss: 0.208702
Train Epoch: 2 [55680/60000 (93%)] Loss: 0.490098
INFO:__main__:Train Epoch: 3 [1280/60000 (2%)] Loss: 0.323959
Train Epoch: 2 [56320/60000 (94%)] Loss: 0.282643
INFO:__main__:Train Epoch: 3 [1920/60000 (3%)] Loss: 0.382809
Train Epoch: 2 [56960/60000 (95%)] Loss: 0.396513
INFO:__main__:Train Epoch: 3 [2560/60000 (4%)] Loss: 0.262482
Train Epoch: 2 [57600/60000 (96%)] Loss: 0.264887
INFO:__main__:Train Epoch: 3 [3200/60000 (5%)] Loss: 0.472320
Train Epoch: 2 [58240/60000 (97%)] Loss: 0.339846
INFO:__main__:Train Epoch: 3 [3840/60000 (6%)] Loss: 0.316821
Train Epoch: 2 [58880/60000

INFO:__main__:Train Epoch: 3 [42240/60000 (70%)] Loss: 0.203819
Train Epoch: 3 [35840/60000 (60%)] Loss: 0.236937
INFO:__main__:Train Epoch: 3 [42880/60000 (71%)] Loss: 0.170989
Train Epoch: 3 [36480/60000 (61%)] Loss: 0.262188
INFO:__main__:Train Epoch: 3 [43520/60000 (72%)] Loss: 0.434066
Train Epoch: 3 [37120/60000 (62%)] Loss: 0.189558
INFO:__main__:Train Epoch: 3 [44160/60000 (74%)] Loss: 0.555165
Train Epoch: 3 [37760/60000 (63%)] Loss: 0.254372
INFO:__main__:Train Epoch: 3 [44800/60000 (75%)] Loss: 0.440353
Train Epoch: 3 [38400/60000 (64%)] Loss: 0.313678
INFO:__main__:Train Epoch: 3 [45440/60000 (76%)] Loss: 0.332281
Train Epoch: 3 [39040/60000 (65%)] Loss: 0.170806
INFO:__main__:Train Epoch: 3 [46080/60000 (77%)] Loss: 0.278145
Train Epoch: 3 [39680/60000 (66%)] Loss: 0.513559
INFO:__main__:Train Epoch: 3 [46080/60000 (77%)] Loss: 0.274961
Train Epoch: 3 [39680/60000 (66%)] Loss: 0.507476
INFO:__main__:Train Epoch: 3 [46720/60000 (78%)] Loss: 0.316760
Train Epoch: 3 [40320/60


2020-02-17 11:19:14 Uploading - Uploading generated training model
2020-02-17 11:19:44 Failed - Training job failed


UnexpectedStatusException: Error for Training job pytorch-training-2020-02-17-11-14-06-031: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python mnist_ckpoint.py --backend nccl --epochs 5 --test-checkpoint 4"
INFO:__main__:Train Epoch: 1 [640/60000 (1%)] Loss: 2.304615
INFO:__main__:Train Epoch: 1 [1280/60000 (2%)] Loss: 2.312131
INFO:__main__:Train Epoch: 1 [1920/60000 (3%)] Loss: 2.291755
INFO:__main__:Train Epoch: 1 [2560/60000 (4%)] Loss: 2.294058
INFO:__main__:Train Epoch: 1 [3200/60000 (5%)] Loss: 2.291983
INFO:__main__:Train Epoch: 1 [3840/60000 (6%)] Loss: 2.259225
INFO:__main__:Train Epoch: 1 [4480/60000 (7%)] Loss: 2.270091
INFO:__main__:Train Epoch: 1 [5120/60000 (9%)] Loss: 2.273199
INFO:__main__:Train Epoch: 1 [5760/60000 (10%)] Loss: 2.240260
INFO:__main__:Train Epoch: 1 [6400/60000 (11%)] Loss: 2.219095
INFO:__main__:Train Epoch: 1 [7040/60000 (12%)] Loss: 2.242404
INFO:__main__:Train Epoch: 1 [7680/60000 (13%)] Loss: 2.145658
INFO:__mai

The original training run failed, as requested. But we have the checkpoints stored in our bucket, under the `/checkpoints` directory. If you check the `mnist_ckpoint.py` script, you'll see that it has two functions that respectively save and restore the checkpoints:
```python
def save_checkpoint(state, filename='/opt/ml/checkpoints/checkpoint.pth.tar'):
    print(f"Saving checkpoint for epoch {state['epoch']} with accuracy {state['best_accuracy']:.3f}")
    torch.save(state, filename)  # save checkpoint

    
def load_checkpoint(filename='/opt/ml/checkpoints/checkpoint.pth.tar'):
    return (torch.load(filename) if os.path.exists(filename) else None)
```

`save_checkpoint` is called inside the training loop, with the following code:
```python
        if (best_accuracy < accuracy) and (hvd.rank() == 0):
            best_accuracy = accuracy
            is_best = True
            save_checkpoint(
                {
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'best_accuracy': best_accuracy
                })
```
The `hvd.rank() == 0` is, according to [horovod's repo](https://github.com/horovod/horovod#usage) (see point 6), the right way to save checkpoints leveraging the framework (the example is for Tensorflow, we adapted it for PyTorch).

`load_checkpoint` is called at the beginning of the `train` function, and if it finds a saved checkpoint training will resume from there:
```python
    start_epoch = 1
    best_accuracy = 0
    checkpoint = load_checkpoint()
    if checkpoint:
        start_epoch = checkpoint['epoch']
        best_accuracy = checkpoint['best_accuracy']
        model.load_state_dict(checkpoint['state_dict'])
```

The next cell creates another PyTorch Estimator, without the request to interrupt training. From the logs, it can be seen that it started from the previous checkpoint

In [ ]:
estimator2 = PyTorch(entry_point='mnist_ckpoint.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.3.1',
                    train_instance_type='ml.p3.2xlarge',
                    metric_definitions=metrics,
                    train_use_spot_instances=True,
                    train_max_wait=25*60*60,
                    train_instance_count=2,
                    checkpoint_s3_uri='s3://sagemaker-us-east-1-113147044314/checkpoints',
                    hyperparameters={
                        'epochs': 5,
                        'backend': 'nccl',
                    })
estimator2.fit({'training': inputs}, wait=True)

2020-02-17 11:21:11 Starting - Starting the training job...
2020-02-17 11:21:13 Starting - Launching requested ML instances......
2020-02-17 11:22:17 Starting - Preparing the instances for training......
2020-02-17 11:23:37 Downloading - Downloading input data...
2020-02-17 11:24:07 Training - Downloading the training image......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-17 11:24:58,460 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-17 11:24:58,485 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-17 11:25:14,877 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-17 11:25:14,902 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2


2020-02-17 11:25:13 Training - Training image download completed. Training in progress.INFO:__main__:Train Epoch: 1 [640/60000 (1%)] Loss: 2.304603
Starting training from the beginning
INFO:__main__:Train Epoch: 1 [1280/60000 (2%)] Loss: 2.312048
Test checkpoint is 0
INFO:__main__:Train Epoch: 1 [1920/60000 (3%)] Loss: 2.291684
Get train data sampler and data loader
INFO:__main__:Train Epoch: 1 [2560/60000 (4%)] Loss: 2.293913
Get test data sampler and data loader
INFO:__main__:Train Epoch: 1 [3200/60000 (5%)] Loss: 2.291727
Processes 60000/60000 (100%) of train data
INFO:__main__:Train Epoch: 1 [3840/60000 (6%)] Loss: 2.258277
Processes 10000/10000 (100%) of test data
INFO:__main__:Train Epoch: 1 [4480/60000 (7%)] Loss: 2.268969
[2020-02-17 11:25:25.250 algo-2:46 INFO json_config.py:90] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
INFO:__main__:Train Epoch: 1 [5120/60000 (9%)] Loss: 2.272753
[2020-02-17 11:25:25.251 algo-2:46 INFO hook.py:152] tensorbo

INFO:__main__:Train Epoch: 1 [18560/60000 (31%)] Loss: 1.008385
Train Epoch: 1 [11520/60000 (19%)] Loss: 1.772861
INFO:__main__:Train Epoch: 1 [19200/60000 (32%)] Loss: 0.814286
Train Epoch: 1 [12160/60000 (20%)] Loss: 1.590020
INFO:__main__:Train Epoch: 1 [19840/60000 (33%)] Loss: 1.060687
Train Epoch: 1 [12800/60000 (21%)] Loss: 1.525059
INFO:__main__:Train Epoch: 1 [20480/60000 (34%)] Loss: 0.714971
Train Epoch: 1 [13440/60000 (22%)] Loss: 1.449087
INFO:__main__:Train Epoch: 1 [21120/60000 (35%)] Loss: 0.588677
Train Epoch: 1 [14080/60000 (23%)] Loss: 1.307881
INFO:__main__:Train Epoch: 1 [21760/60000 (36%)] Loss: 0.904387
Train Epoch: 1 [14720/60000 (25%)] Loss: 0.953352
INFO:__main__:Train Epoch: 1 [22400/60000 (37%)] Loss: 0.930393
Train Epoch: 1 [15360/60000 (26%)] Loss: 1.087243
INFO:__main__:Train Epoch: 1 [23040/60000 (38%)] Loss: 0.949185
Train Epoch: 1 [16000/60000 (27%)] Loss: 1.146010
INFO:__main__:Train Epoch: 1 [44160/60000 (74%)] Loss: 0.720959
Train Epoch: 1 [37120/60

INFO:__main__:Epoch: 1#011Test set: Average loss: 0.2008, Accuracy: 94.02%
Train Epoch: 1 [54400/60000 (91%)] Loss: 0.391875

Train Epoch: 1 [55040/60000 (92%)] Loss: 0.704062
INFO:__main__:Train Epoch: 2 [640/60000 (1%)] Loss: 0.685819
Train Epoch: 1 [55680/60000 (93%)] Loss: 0.466761
INFO:__main__:Train Epoch: 2 [19840/60000 (33%)] Loss: 0.426466
Train Epoch: 2 [13440/60000 (22%)] Loss: 0.503925
INFO:__main__:Train Epoch: 2 [20480/60000 (34%)] Loss: 0.175942
Train Epoch: 2 [14080/60000 (23%)] Loss: 0.431281
INFO:__main__:Train Epoch: 2 [21120/60000 (35%)] Loss: 0.312501
Train Epoch: 2 [14720/60000 (25%)] Loss: 0.174918
INFO:__main__:Train Epoch: 2 [21760/60000 (36%)] Loss: 0.598874
Train Epoch: 2 [15360/60000 (26%)] Loss: 0.334293
INFO:__main__:Train Epoch: 2 [22400/60000 (37%)] Loss: 0.416368
Train Epoch: 2 [16000/60000 (27%)] Loss: 0.521367
INFO:__main__:Train Epoch: 2 [23040/60000 (38%)] Loss: 0.443982
Train Epoch: 2 [16640/60000 (28%)] Loss: 0.491021
INFO:__main__:Train Epoch: 2 

INFO:__main__:Train Epoch: 2 [42880/60000 (71%)] Loss: 0.372698
Train Epoch: 2 [36480/60000 (61%)] Loss: 0.419038
INFO:__main__:Train Epoch: 2 [43520/60000 (72%)] Loss: 0.449386
Train Epoch: 2 [37120/60000 (62%)] Loss: 0.403179
INFO:__main__:Train Epoch: 2 [44160/60000 (74%)] Loss: 0.433247
Train Epoch: 2 [37760/60000 (63%)] Loss: 0.301032
INFO:__main__:Train Epoch: 2 [44800/60000 (75%)] Loss: 0.593096
Train Epoch: 2 [38400/60000 (64%)] Loss: 0.433173
INFO:__main__:Train Epoch: 2 [45440/60000 (76%)] Loss: 0.244111
Train Epoch: 2 [39040/60000 (65%)] Loss: 0.327437
INFO:__main__:Train Epoch: 2 [46080/60000 (77%)] Loss: 0.375437
Train Epoch: 2 [39680/60000 (66%)] Loss: 0.384601
INFO:__main__:Train Epoch: 2 [46720/60000 (78%)] Loss: 0.463178
Train Epoch: 2 [40320/60000 (67%)] Loss: 0.561495
INFO:__main__:Train Epoch: 2 [47360/60000 (79%)] Loss: 0.486274
Train Epoch: 2 [40960/60000 (68%)] Loss: 0.443896
INFO:__main__:Epoch: 2#011Test set: Average loss: 0.1271, Accuracy: 96.11%
Train Epoch: 

INFO:__main__:Train Epoch: 3 [20480/60000 (34%)] Loss: 0.327778
Train Epoch: 3 [13440/60000 (22%)] Loss: 0.356703
INFO:__main__:Train Epoch: 3 [21120/60000 (35%)] Loss: 0.256373
Train Epoch: 3 [14080/60000 (23%)] Loss: 0.398225
INFO:__main__:Train Epoch: 3 [21760/60000 (36%)] Loss: 0.449268
Train Epoch: 3 [14720/60000 (25%)] Loss: 0.152495
INFO:__main__:Train Epoch: 3 [22400/60000 (37%)] Loss: 0.459165
Train Epoch: 3 [15360/60000 (26%)] Loss: 0.167592
INFO:__main__:Train Epoch: 3 [23040/60000 (38%)] Loss: 0.231470
Train Epoch: 3 [16000/60000 (27%)] Loss: 0.273813
INFO:__main__:Train Epoch: 3 [23680/60000 (39%)] Loss: 0.353869
Train Epoch: 3 [16640/60000 (28%)] Loss: 0.394751
INFO:__main__:Train Epoch: 3 [24320/60000 (41%)] Loss: 0.305309
Train Epoch: 3 [17280/60000 (29%)] Loss: 0.159762
INFO:__main__:Train Epoch: 3 [40960/60000 (68%)] Loss: 0.291024
Train Epoch: 3 [33920/60000 (57%)] Loss: 0.183516
INFO:__main__:Train Epoch: 3 [41600/60000 (69%)] Loss: 0.297025
Train Epoch: 3 [34560/60

INFO:__main__:Epoch: 3#011Test set: Average loss: 0.1001, Accuracy: 96.79%
Train Epoch: 3 [53120/60000 (88%)] Loss: 0.492994

Train Epoch: 3 [53760/60000 (90%)] Loss: 0.331074
INFO:__main__:Train Epoch: 4 [640/60000 (1%)] Loss: 0.390152
Train Epoch: 3 [54400/60000 (91%)] Loss: 0.143555
INFO:__main__:Train Epoch: 4 [1280/60000 (2%)] Loss: 0.274002
Train Epoch: 3 [55040/60000 (92%)] Loss: 0.413738
INFO:__main__:Train Epoch: 4 [1920/60000 (3%)] Loss: 0.184563
Train Epoch: 3 [55680/60000 (93%)] Loss: 0.374394
INFO:__main__:Train Epoch: 4 [2560/60000 (4%)] Loss: 0.316577
Train Epoch: 3 [56320/60000 (94%)] Loss: 0.105711
INFO:__main__:Train Epoch: 4 [3200/60000 (5%)] Loss: 0.321438
Train Epoch: 3 [56960/60000 (95%)] Loss: 0.261124
INFO:__main__:Train Epoch: 4 [3840/60000 (6%)] Loss: 0.336956
Train Epoch: 3 [57600/60000 (96%)] Loss: 0.184127
INFO:__main__:Train Epoch: 4 [4480/60000 (7%)] Loss: 0.236679
Train Epoch: 3 [58240/60000 (97%)] Loss: 0.094730
INFO:__main__:Train Epoch: 4 [5120/60000 

INFO:__main__:Train Epoch: 4 [44800/60000 (75%)] Loss: 0.424614
Train Epoch: 4 [37120/60000 (62%)] Loss: 0.196371
INFO:__main__:Train Epoch: 4 [45440/60000 (76%)] Loss: 0.194445
Train Epoch: 4 [37760/60000 (63%)] Loss: 0.345195
INFO:__main__:Train Epoch: 4 [46080/60000 (77%)] Loss: 0.142868
Train Epoch: 4 [38400/60000 (64%)] Loss: 0.368374
INFO:__main__:Train Epoch: 4 [46720/60000 (78%)] Loss: 0.253209
Train Epoch: 4 [39040/60000 (65%)] Loss: 0.394422
INFO:__main__:Train Epoch: 4 [47360/60000 (79%)] Loss: 0.268019
Train Epoch: 4 [39680/60000 (66%)] Loss: 0.303954
INFO:__main__:Train Epoch: 4 [48000/60000 (80%)] Loss: 0.272374
Train Epoch: 4 [40320/60000 (67%)] Loss: 0.174165
INFO:__main__:Train Epoch: 4 [48640/60000 (81%)] Loss: 0.141553
Train Epoch: 4 [40960/60000 (68%)] Loss: 0.226078
INFO:__main__:Train Epoch: 4 [49280/60000 (82%)] Loss: 0.209236
Train Epoch: 4 [41600/60000 (69%)] Loss: 0.350925
INFO:__main__:Train Epoch: 4 [49920/60000 (83%)] Loss: 0.682645
Train Epoch: 4 [42240/60

INFO:__main__:Train Epoch: 5 [21120/60000 (35%)] Loss: 0.170458
Train Epoch: 5 [12800/60000 (21%)] Loss: 0.271309
INFO:__main__:Train Epoch: 5 [21760/60000 (36%)] Loss: 0.579850
Train Epoch: 5 [13440/60000 (22%)] Loss: 0.236044
INFO:__main__:Train Epoch: 5 [22400/60000 (37%)] Loss: 0.243769
Train Epoch: 5 [14080/60000 (23%)] Loss: 0.261704
INFO:__main__:Train Epoch: 5 [23040/60000 (38%)] Loss: 0.338645
Train Epoch: 5 [14720/60000 (25%)] Loss: 0.147323
INFO:__main__:Train Epoch: 5 [23680/60000 (39%)] Loss: 0.222933
Train Epoch: 5 [15360/60000 (26%)] Loss: 0.172694
INFO:__main__:Train Epoch: 5 [24320/60000 (41%)] Loss: 0.267527
Train Epoch: 5 [16000/60000 (27%)] Loss: 0.289326
INFO:__main__:Train Epoch: 5 [24960/60000 (42%)] Loss: 0.188442
Train Epoch: 5 [16640/60000 (28%)] Loss: 0.220537
INFO:__main__:Train Epoch: 5 [25600/60000 (43%)] Loss: 0.219004
Train Epoch: 5 [17280/60000 (29%)] Loss: 0.180635
INFO:__main__:Train Epoch: 5 [26240/60000 (44%)] Loss: 0.167003
Train Epoch: 5 [17920/60

INFO:__main__:Epoch: 5#011Test set: Average loss: 0.0749, Accuracy: 97.58%
Train Epoch: 5 [51840/60000 (86%)] Loss: 0.245618

Train Epoch: 5 [52480/60000 (87%)] Loss: 0.185763
Train Epoch: 5 [53120/60000 (88%)] Loss: 0.543138
Train Epoch: 5 [53760/60000 (90%)] Loss: 0.254244
Train Epoch: 5 [54400/60000 (91%)] Loss: 0.130588
Train Epoch: 5 [55040/60000 (92%)] Loss: 0.340800
Train Epoch: 5 [55680/60000 (93%)] Loss: 0.325343
Train Epoch: 5 [56320/60000 (94%)] Loss: 0.103922
Train Epoch: 5 [56960/60000 (95%)] Loss: 0.303123
Train Epoch: 5 [57600/60000 (96%)] Loss: 0.194148
Train Epoch: 5 [58240/60000 (97%)] Loss: 0.253291
Train Epoch: 5 [58880/60000 (98%)] Loss: 0.275624
Train Epoch: 5 [59520/60000 (99%)] Loss: 0.271151
Epoch: 5#011Test set: Average loss: 0.0749, Accuracy: 97.58%

Saving checkpoint for epoch 5 with accuracy 0.976
[2020-02-17 11:26:48.638 algo-1:46 INFO utils.py:25] The end of training job file will not be written for jobs running under SageMaker.
2020-02-17 11:26:49,012 sa

In [ ]:
print("\n".join([f"{m['MetricName']}: {m['Value']}"for m in estimator2.latest_training_job.describe()['FinalMetricDataList']]))

Test Accuracy: 97.58000183105469
Test Loss: 0.07490000128746033
Epoch: 5.0
Train loss: 0.27115100622177124


Now that we have a fully trained estimator, we could save it as a deployable model. But before we do so, let's see how we can fine-tune the hyperparameters for it.

## Hyperparameter Tuning

In [ ]:
hpo_start_time = datetime.now()

The first step for hyperparameter optimization is to define which parameters should be tried and their respective ranges or categories. We'll showcase how it's done with the learning rate and the batch size, but any parameter can be tried. Actually, by adapting the training script and passing it some parameters, even the network architecture itself could be tried dynamically.

We'll also define which metric we want to optimize for, in this case average test loss.

In [ ]:
hyperparameter_ranges = {
    'lr': ContinuousParameter(0.001, 0.1),
    'batch-size': CategoricalParameter([32,64,128])
}
objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Test set: Average loss: ([0-9\\.]+)'}]

With the parameter ranges and target metric ready, we create a tuner from the estimator we trained before. We are limiting the search to 9 attempts and telling it to try up to 3 parallel training optimizations. Note that your account has to have quotas for the combination of optimization runs **times** the degree of parallelism of the estimator, and that the Bayesian optimization will have a better chance to reduce the total cost when trials are run sequentially.

In [ ]:
tuner = HyperparameterTuner(estimator2,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            early_stopping_type="Off",
                            objective_type=objective_type)

In [ ]:
tuner.fit({'training': inputs})

In [17]:
tuner.wait()

.....................................................................................................................................................................................................................!


Once tuning is finished, we can request the best estimator and check its hyperparameters and achieved results.

In [18]:
best = tuner.best_estimator()

2020-02-17 11:40:37 Starting - Preparing the instances for training
2020-02-17 11:40:37 Downloading - Downloading input data
2020-02-17 11:40:37 Training - Training image download completed. Training in progress.
2020-02-17 11:40:37 Uploading - Uploading generated training model
2020-02-17 11:40:37 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-17 11:38:56,381 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-17 11:38:56,384 sagemaker-containers INFO     Failed to parse hyperparameter _tuning_objective_metric value average test loss to Json.
Returning the value itself
2020-02-17 11:38:56,409 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-02-17 11:38:57,825 sagemaker_pytorch_containe

In [19]:
best.hyperparameters()

{'_tuning_objective_metric': '"average test loss"',
 'backend': '"nccl"',
 'batch-size': '"128"',
 'epochs': '5',
 'lr': '0.07896950695095702',
 'sagemaker_container_log_level': '20',
 'sagemaker_enable_cloudwatch_metrics': 'false',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2020-02-17-11-28-26-417"',
 'sagemaker_program': '"mnist_ckpoint.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-113147044314/pytorch-training-2020-02-17-11-28-26-417/source/sourcedir.tar.gz"'}

In [20]:
print("\n".join([f"{m['MetricName']}: {m['Value']}"for m in best.latest_training_job.describe()['FinalMetricDataList']]))

average test loss: 0.05079999938607216
ObjectiveMetric: 0.05079999938607216


## Host
### Create endpoint
After training, we need to use the `PyTorch` estimator object to create a `PyTorchModel` object and set a different `entry_point`, otherwise, the training script `mnist_ckpoint.py` will be used for inference. (Note that the new `entry_point` must be under the same `source_dir` as `mnist_ckpoint.py`). Then we use the `PyTorchModel` object to deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

An implementation of `model_fn` is required for inference script. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

Here's an example of the inference script:

In [21]:
!pygmentize code/inference.py

from __future__ import print_function
import os
import torch

from model_def import Net


def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Net()
    with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
        model.load_state_dict(torch.load(f))
    return model.to(device)


The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job.  Here we will deploy the model to a single ```ml.p3.2xlarge``` instance. Notice that endpoint deployment can take up to 10 minutes.

In [22]:
deploy_start_time = datetime.now()

In [23]:
# Create a PyTorchModel object with a different entry_point
model = best.create_model(entry_point='inference.py', source_dir='code')

In [24]:
# Deploy the model to an instance
predictor_gpu = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge', wait=True)

-------------------!

In [25]:
print("Endpoint characteristics:\n"
      f"\tAccept: {predictor_gpu.accept}\n"
      f"\tContent Type: {predictor_gpu.content_type}\n"
      f"\tSerializer: {predictor_gpu.serializer}\n"
      f"\tDeserializer: {predictor_gpu.deserializer}\n"
      f"\tEndpoint: {predictor_gpu.endpoint}")

Endpoint characteristics:
	Accept: application/x-npy
	Content Type: application/x-npy
	Serializer: <sagemaker.predictor._NPYSerializer object at 0x7fb924f0ea58>
	Deserializer: <sagemaker.predictor._NumpyDeserializer object at 0x7fb924f0e898>
	Endpoint: pytorch-training-200217-1128-004-d84ef244


### Evaluate
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`. Notice that the drawing box only works under the original Jupyter interface (not Jupyter Lab).

In [26]:
eval_start_time = datetime.now()

In [27]:
HTML(open("input.html").read())

In [28]:
%%time
image = np.array([data], dtype=np.float32)
response = predictor_gpu.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

2
CPU times: user 7.47 ms, sys: 4.08 ms, total: 11.6 ms
Wall time: 5.84 s


#### Predictor Timing

We can also take some measures of how efficient each instance type is. Given that the model itself is quite simple, we should not see much difference between GPU and CPU powered instances.

In [33]:
%%timeit
results = predictor_gpu.predict(image)

11.2 ms ± 837 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
predictor_gpu.delete_endpoint()
time.sleep(5)

In [35]:
predictor_cpu = model.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge', wait=True)

-------------------!

In [36]:
%%timeit
results = [predictor_cpu.predict(image) for _ in range(100)]

1.06 s ± 61.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
predictor_cpu.delete_endpoint()

Always remember to delete the endpoints after trying them out. Otherwise they'll remain active and generate additional costs in your account (even if idle).

In [38]:
end_time = datetime.now()

In [39]:
total_duration = end_time - start_time
data_duration = train_start_time - data_start_time
train_duration = hpo_start_time - train_start_time
hpo_duration = deploy_start_time - hpo_start_time
deploy_duration = eval_start_time - deploy_start_time

print(f"Total: {total_duration}\nData Load: {data_duration}\nTraining: {train_duration}\nHPO: {hpo_duration}\nDeploy: {deploy_duration}")

Total: 1:12:28.351083
Data Load: 0:00:04.658180
Training: 0:14:20.437503
HPO: 0:19:08.737477
Deploy: 0:09:32.900177
